In [5]:
# Cell 1: Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from tqdm import tqdm # For a progress bar

# Add the 'src' folder to the path
sys.path.insert(0, os.path.abspath('../src'))

# --- Import our two OU classes ---
from models.ou import OrnsteinUhlenbeck
from calibration.ou_calibrator import OuCalibrator

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
print("All modules imported successfully.")

All modules imported successfully.


In [6]:
# Cell 2: Setup the Monte Carlo Test
# These are the "true" parameters we want to find
true_ou_params = {
    'X0': 20,         # Start value
    'mu': 20.0,       # 20 is the long-term mean (like VIX)
    'theta': 5.0,     # 5.0 is a strong reversion speed
    'sigma': 5.0      # 5.0 is the volatility
}

# --- Simulation setup ---
# We need a long time period for OU to be stable
T = 10.0      # 10 years of data for each test
dt = 1/252    # Daily steps
n_test_paths = 100 # Run 100 tests to get a good average

# --- Result storage ---
calibrated_thetas = []
calibrated_mus = []
calibrated_sigmas = []

# --- Instantiate our models once ---
ou_model = OrnsteinUhlenbeck(**true_ou_params)
ou_cal = OuCalibrator(dt=dt)

print(f"Running {n_test_paths} OU calibration tests...")
print(f"True Params: {true_ou_params}")

Running 100 OU calibration tests...
True Params: {'X0': 20, 'mu': 20.0, 'theta': 5.0, 'sigma': 5.0}


In [7]:
# Cell 3: Run the Test Loop
# This cell should be very fast

for _ in tqdm(range(n_test_paths)):
    # 1. Simulate one new, random path
    sim_path_array = ou_model.simulate(T, dt, n_paths=1).flatten()
    sim_series = pd.Series(sim_path_array)

    # 2. Calibrate this one path
    params = ou_cal.fit(sim_series)
    
    # 3. Store the results
    calibrated_thetas.append(params.get('theta', np.nan))
    calibrated_mus.append(params.get('mu', np.nan))
    calibrated_sigmas.append(params.get('sigma', np.nan))

print(f"--- Test Complete ---")

100%|██████████| 100/100 [00:01<00:00, 87.78it/s]

--- Test Complete ---


In [8]:
# Cell 4: Analyze and Display Results
# 1. Get the average of each parameter column
mean_theta = np.nanmean(calibrated_thetas)
mean_mu = np.nanmean(calibrated_mus)
mean_sigma = np.nanmean(calibrated_sigmas)

# 2. Get the standard deviation of our estimates
std_theta = np.nanstd(calibrated_thetas)
std_mu = np.nanstd(calibrated_mus)
std_sigma = np.nanstd(calibrated_sigmas)

# 3. Print a clean comparison table
print("\n=== True vs Average Calibrated Parameters ===")
print(f"{'Parameter':<16}{'True Value':>15}{'Average Calibrated':>25}")
print("-" * 56)
print(f"{'θ (reversion)':<16}{true_ou_params['theta']:>15.4f}{mean_theta:>25.4f}")
print(f"{'μ (mean)':<16}{true_ou_params['mu']:>15.4f}{mean_mu:>25.4f}")
print(f"{'σ (volatility)':<16}{true_ou_params['sigma']:>15.4f}{mean_sigma:>25.4f}")

# 4. Print the standard deviation of the estimates
print("\n=== Estimation Standard Deviations ===")
print(f"{'Parameter':<16}{'Std Dev (across runs)':>30}")
print("-" * 46)
print(f"{'θ (reversion)':<16}{std_theta:>30.4f}")
print(f"{'μ (mean)':<16}{std_mu:>30.4f}")
print(f"{'σ (volatility)':<16}{std_sigma:>30.4f}")


=== True vs Average Calibrated Parameters ===
Parameter            True Value       Average Calibrated
--------------------------------------------------------
θ (reversion)            5.0000                   5.4436
μ (mean)                20.0000                  19.9404
σ (volatility)           5.0000                   4.9997

=== Estimation Standard Deviations ===
Parameter                Std Dev (across runs)
----------------------------------------------
θ (reversion)                           0.9893
μ (mean)                                0.3360
σ (volatility)                          0.0709
